In [42]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [43]:
import warnings
warnings.filterwarnings('ignore')

In [44]:
df = pd.read_excel(r"C:\Users\sivak\Downloads\students_assessment.xlsx", index_col=[0, 1,2], header=[0, 1])
df.head()

ENGLISH                          \
TRACK                                      PP1_60 PP2_80 PP3_60   AVE GRD   
PALOMA ESTHER ANINDO 11235127001 VICTORIA    10.0   26.0   18.0  27.0   E   
ANGIE WACERA SAKWA   11235127002 CHALA       19.0   43.0   27.0  44.5  D+   
NJUGUNA NADIA NJAMBI 11235127003 VICTORIA    22.0   45.0   18.0  42.5  D+   
THUO IMANI WANJIRU   11235127004 SOLAI       25.0   39.0   31.0  47.5  C-   
ELIJAH CHERIE NEEMA  11235127005 VICTORIA    26.0   42.0   26.0  47.0  C-   

                                               KISWAHILI                      \
TRACK                                      PTS    PP1_40 PP2_80 PP3_80   AVE   
PALOMA ESTHER ANINDO 11235127001 VICTORIA  1.0      22.0     26   19.0  33.5   
ANGIE WACERA SAKWA   11235127002 CHALA     4.0      26.0     55   53.0  67.0   
NJUGUNA NADIA NJAMBI 11235127003 VICTORIA  4.0      25.0     58   54.0  68.5   
THUO IMANI WANJIRU   11235127004 SOLAI     5.0      25.0     41   55.0  60.5   
ELIJAH CHERIE NEEMA  11235127005 VICTORIA  5.0      28.0     55   49.0  66.0   

                                           ...   C/st              BST  \
TRACK                                      ...    AVE GRD  PTS PP1_100   
PALOMA ESTHER ANINDO 11235127001 VICTORIA  ...  45.00  D+  4.0     NaN   
ANGIE WACERA SAKWA   11235127002 CHALA     ...  70.75  B-  8.0     NaN   
NJUGUNA NADIA NJAMBI 11235127003 VICTORIA  ...  55.75  C-  5.0     NaN   
THUO IMANI WANJIRU   11235127004 SOLAI     ...  50.50  C-  5.0    57.0   
ELIJAH CHERIE NEEMA  11235127005 VICTORIA  ...  61.25   C  6.0     NaN   

                                                                   TOTAL  \
TRACK                                     PP2_100   AVE  GRD  PTS POINTS   
PALOMA ESTHER ANINDO 11235127001 VICTORIA     NaN   NaN  NaN  NaN   14.0   
ANGIE WACERA SAKWA   11235127002 CHALA        NaN   NaN  NaN  NaN   39.0   
NJUGUNA NADIA NJAMBI 11235127003 VICTORIA     NaN   NaN  NaN  NaN   34.0   
THUO IMANI WANJIRU   11235127004 SOLAI       38.0  47.5   D+  4.0   30.0   
ELIJAH CHERIE NEEMA  11235127005 VICTORIA     NaN   NaN  NaN  NaN   42.0   

                                           GRADE  
TRACK                                     POINTS  
PALOMA ESTHER ANINDO 11235127001 VICTORIA     D-  
ANGIE WACERA SAKWA   11235127002 CHALA         C  
NJUGUNA NADIA NJAMBI 11235127003 VICTORIA     C-  
THUO IMANI WANJIRU   11235127004 SOLAI        D+  
ELIJAH CHERIE NEEMA  11235127005 VICTORIA      C  

[5 rows x 62 columns]

In [45]:
df = df.fillna(0)

In [46]:
df.columns = ['_'.join(col).strip() if col[1] else col[0] for col in df.columns.values]

In [47]:
df.columns = [''.join(filter(None, col)).strip() for col in df.columns.values]

In [48]:
df.reset_index(inplace=True)

In [49]:
df.rename(columns={'level_0':"STUDENT NAME", "level_1":"INDEX NUMBER", "level_2":"TRACK"}, inplace=True)

In [50]:
df.head()

,STUDENT NAME,INDEX NUMBER,TRACK,ENGLISH_PP1_60,ENGLISH_PP2_80,ENGLISH_PP3_60,ENGLISH_AVE,ENGLISH_GRD,ENGLISH_PTS,KISWAHILI_PP1_40,...,C/st_AVE,C/st_GRD,C/st_PTS,BST_PP1_100,BST_PP2_100,BST_AVE,BST_GRD,BST_PTS,TOTAL_POINTS,GRADE_POINTS
0,PALOMA ESTHER ANINDO,11235127001,VICTORIA,10.0,26.0,18.0,27.0,E,1.0,22.0,...,45.00,D+,4.0,0.0,0.0,0.0,0,0.0,14.0,D-
1,ANGIE WACERA SAKWA,11235127002,CHALA,19.0,43.0,27.0,44.5,D+,4.0,26.0,...,70.75,B-,8.0,0.0,0.0,0.0,0,0.0,39.0,C
2,NJUGUNA NADIA NJAMBI,11235127003,VICTORIA,22.0,45.0,18.0,42.5,D+,4.0,25.0,...,55.75,C-,5.0,0.0,0.0,0.0,0,0.0,34.0,C-
3,THUO IMANI WANJIRU,11235127004,SOLAI,25.0,39.0,31.0,47.5,C-,5.0,25.0,...,50.50,C-,5.0,57.0,38.0,47.5,D+,4.0,30.0,D+
4,ELIJAH CHERIE NEEMA,11235127005,VICTORIA,26.0,42.0,26.0,47.0,C-,5.0,28.0,...,61.25,C,6.0,0.0,0.0,0.0,0,0.0,42.0,C


In [51]:
subjects = {
    "ENGLISH": ["ENGLISH_PP1_60", "ENGLISH_PP2_80", "ENGLISH_PP3_60", "ENGLISH_AVE", "ENGLISH_PTS"],
    "KISWAHILI": ["KISWAHILI_PP1_40", "KISWAHILI_PP2_80", "KISWAHILI_PP3_80", "KISWAHILI_AVE", "KISWAHILI_PTS"],
    "MATH": ["MATH_PP1_100", "MATH_PP2_100", "MATH_AVE", "MATH_PTS"],
    "BIOLOGY": ["BIOLOGY_PP1_80", "BIOLOGY_PP2_80", "BIOLOGY_PP3_40", "BIOLOGY_AVE", "BIOLOGY_PTS"],
    "PHYSICS": ["PHYSICS_PP1_80", "PHYSICS_PP2_80", "PHYSICS_PP3_40", "PHYSICS_AVE", "PHYSICS_PTS"],
    "CHEMISTRY": ["CHEMISTRY_PP1_80", "CHEMISTRY_PP2_80", "CHEMISTRY_PP3_40", "CHEMISTRY_AVE", "CHEMISTRY_PTS"],
    "HIST": ["HIST_PP1_100", "HIST_PP2_100", "HIST_AVE", "HIST_PTS"],
    "GEO": ["GEO_PP1_100", "GEO_PP2_100", "GEO_AVE", "GEO_PTS"],
    "CRE": ["CRE_PP1_100", "CRE_PP2_100", "CRE_AVE", "CRE_PTS"],
    "C/st": ["C/st_PP1_100", "C/st_PP2_100", "C/st_AVE", "C/st_PTS"],
    "BST": ["BST_PP1_100", "BST_PP2_100", "BST_AVE", "BST_PTS"],
}

for subject, columns in subjects.items():
    for col in columns:
        if col in df.columns:
            df[col] = df[col].astype(int)
        else:
            print(f"Column {col} does not exist in the DataFrame for {subject}.")

print("Conversion complete.")


Conversion complete.


In [81]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 65 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   STUDENT NAME      95 non-null     object 
 1   INDEX NUMBER      95 non-null     int64  
 2   TRACK             95 non-null     object 
 3   ENGLISH_PP1_60    95 non-null     int32  
 4   ENGLISH_PP2_80    95 non-null     int32  
 5   ENGLISH_PP3_60    95 non-null     int32  
 6   ENGLISH_AVE       95 non-null     int32  
 7   ENGLISH_GRD       95 non-null     object 
 8   ENGLISH_PTS       95 non-null     int32  
 9   KISWAHILI_PP1_40  95 non-null     int32  
 10  KISWAHILI_PP2_80  95 non-null     int32  
 11  KISWAHILI_PP3_80  95 non-null     int32  
 12  KISWAHILI_AVE     95 non-null     int32  
 13  KISWAHILI_GRD     95 non-null     object 
 14  KISWAHILI_PTS     95 non-null     int32  
 15  MATH_PP1_100      95 non-null     int32  
 16  MATH_PP2_100      95 non-null     int32  
 17 

In [83]:
df.isna().sum()

STUDENT NAME      0
INDEX NUMBER      0
TRACK             0
ENGLISH_PP1_60    0
ENGLISH_PP2_80    0
                 ..
BST_AVE           0
BST_GRD           0
BST_PTS           0
TOTAL_POINTS      0
GRADE_POINTS      0
Length: 65, dtype: int64

In [85]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
INDEX NUMBER,95.0,1.123513e+10,27.550146,1.123513e+10,1.123513e+10,1.123513e+10,1.123513e+10,1.123513e+10
ENGLISH_PP1_60,95.0,2.303158e+01,5.153913,1.000000e+01,1.900000e+01,2.300000e+01,2.600000e+01,3.400000e+01
ENGLISH_PP2_80,95.0,3.742105e+01,7.529487,2.300000e+01,3.250000e+01,3.800000e+01,4.200000e+01,5.600000e+01
ENGLISH_PP3_60,95.0,2.798947e+01,6.083628,1.300000e+01,2.400000e+01,2.800000e+01,3.100000e+01,4.200000e+01
ENGLISH_AVE,95.0,4.394737e+01,7.529189,2.500000e+01,3.850000e+01,4.500000e+01,5.000000e+01,6.000000e+01
ENGLISH_PTS,95.0,4.284211e+00,1.541367,1.000000e+00,3.000000e+00,4.000000e+00,5.500000e+00,7.000000e+00
KISWAHILI_PP1_40,95.0,2.492632e+01,3.437101,1.100000e+01,2.300000e+01,2.600000e+01,2.700000e+01,3.100000e+01
KISWAHILI_PP2_80,95.0,4.581053e+01,10.603386,1.000000e+01,3.900000e+01,4.600000e+01,5.400000e+01,6.600000e+01
KISWAHILI_PP3_80,95.0,4.322105e+01,12.534360,1.300000e+01,3.400000e+01,4.400000e+01,5.300000e+01,6.800000e+01
KISWAHILI_AVE,95.0,5.673684e+01,11.612732,1.800000e+01,4.900000e+01,5.800000e+01,6.650000e+01,7.900000e+01


In [52]:
df['GRADE_POINTS'] = df['GRADE_POINTS'].astype(str)

In [53]:
subject_columns = ['ENGLISH_AVE', 'KISWAHILI_AVE', 'MATH_AVE', 'BIOLOGY_AVE', 'PHYSICS_AVE', 
                   'CHEMISTRY_AVE', 'HIST_AVE', 'GEO_AVE', 'CRE_AVE', 'C/st_AVE', 'BST_AVE']

X = df[subject_columns]

label_encoder = LabelEncoder()   # Label encode on "GRADE_POINTS"
y = label_encoder.fit_transform(df['GRADE_POINTS'])

# One-hot encode the labels
y = to_categorical(y)

In [54]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    Dense(y.shape[1], activation='softmax')
])

In [55]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y, epochs=150, batch_size=32)

Epoch 1/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.0589 - loss: 14.2583
Epoch 2/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2029 - loss: 5.0675 
Epoch 3/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1846 - loss: 3.7790 
Epoch 4/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1455 - loss: 3.2014 
Epoch 5/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2669 - loss: 2.4584 
Epoch 6/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2787 - loss: 2.1091 
Epoch 7/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2919 - loss: 1.8357 
Epoch 8/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.2841 - loss: 1.8838 
Epoch 9/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.1926 - loss: 1.7945 
Epoch 10/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3167 - loss: 1.6281 
Epoch 11/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.3456 - loss: 1.5470 
Epoch 12/150
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.3559 - los

### Function to recommend subjects for improvement

In [57]:
def recommend_subjects(student_id, threshold=50):
    student_scores = df[df['INDEX NUMBER'] == student_id][subject_columns].values
    student_scores = student_scores.reshape(1, -1)

    grade = model.predict(student_scores)  # Predict grade secured by student
    grade_label = label_encoder.inverse_transform([grade.argmax()])
    print(f"Grade secured by student {student_id}: {grade_label[0]}")

    student_scores = df[df['INDEX NUMBER'] == student_id][subject_columns].values.flatten()  # Get the actual scores of the student

    low_score_subjects = [subject_columns[i] for i, score in enumerate(student_scores) if score < threshold]  # Subjects with scores below the threshold

    if low_score_subjects:
        print(f"Subjects to improve for student {student_id}: {', '.join(low_score_subjects)}")
    else:
        print(f"Student {student_id} is performing well in all subjects!")

In [87]:
# Test with a specific student
student_id = 11235127001
recommend_subjects(student_id, threshold=50)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
Grade secured by student 11235127001: D-
Subjects to improve for student 11235127001: ENGLISH_AVE, KISWAHILI_AVE, MATH_AVE, BIOLOGY_AVE, PHYSICS_AVE, CHEMISTRY_AVE, HIST_AVE, GEO_AVE, CRE_AVE, C/st_AVE, BST_AVE
